In [1]:
from fitting import FitModel
from databases import initiate_dbs, print_summary, print_forces
from plotters import pymc3_trace_plots, forces_plots
from iteration_functions import iter_fitting, update_potentials
from convergence_checker import get_modes, converge_check
from scipy import stats

In [2]:
params = {}
params['core_shell'] = { 'Li': False, 'Ni': False, 'O': True }
params['charges'] = {'Li': +1.0,
                     'Ni': +3.0,
                     'O': {'core':  +0.960,
                           'shell': -2.960}}
params['masses'] = {'Li': 6.941,
                    'Ni': 58.6934,
                    'O': {'core': 14.3991,
                          'shell': 1.5999} }

params['cs_springs'] = {'O' : [100.0, 0.0]}

distribution = {}
distribution['Li-O'] = {'bpp' : [691.229, 0.269, 0.0],
                        'sd' : [80, 0.01, 0.01]}

distribution['Ni-O'] = {'bpp' : [591.665, 0.382, 0.000],
                        'sd'  : [80, 0.01, 0.01]}

distribution['O-O'] = {'bpp' : [22739.211, 0.146, 20.7],
                       'sd'  : [200, 0.01, 5]}

excude_from_fit = [] # string of atom1_atom2_param. Example of format = 'O_O_rho'

In [3]:
def mode_potentials(trace):
    potential_dic = {}
    for var in trace.varnames:
        potential_dic['{}'.format(var)] = float(stats.mode(trace.get_values(var))[0])
    return potential_dic

In [ ]:
i=1
prev_modes = None
converge = False
summary_filename = 'summary.csv'
forces_filename = 'forces.csv'
initiate_dbs(summary_filename, forces_filename)

while converge is False:
    
    #Runs FitModel and trace, finds mode, and updates the potentials in the distribution dictionary with mean values
    trace, fit_data = iter_fitting(params, distribution, excude_from_fit)
    modes = get_modes(trace)
    distribution = update_potentials(trace, modes, distribution)
    
    #Runs with mode potential and returns forces
    kwargs = mode_potentials(trace)
    mode_forces = fit_data.get_forces(**kwargs)
    
    #Fills the databases
    print_forces(i, mode_forces, forces_filename)
    print_summary(i, trace, summary_filename)
    
    #Plots the distributions
    pymc3_trace_plots(trace, i)
    forces_plots(forces_filename, i)
    
    #Checks convergence, sets modes to prev_moves
    converge = converge_check(modes, distribution, prev_modes)
    prev_modes = modes
    i+=1

print('converged')

In [11]:
import pandas as pd
import re
import numpy as np

In [4]:
i=1
summary_filename = 'summary.csv'
forces_filename = 'forces.csv'

In [54]:
def update_potentials(sd, modes, distribution):
    """
    Updates the potentials in the distribution dictionary to the mode values calculated from the distribution and the standard deviations output by the fit in the trace, ready for the next iteration.
    Args:
        trace (obj): A pymc3.backends.base.MultiTrace object containing a multitrace with information on the number of chains, iterations, and variables output from PyMC3.
        modes (list(float)): Mode potential parameter values for current iteration.
        distribution(dict(dict)): Contains buckingham potential, 'bpp':list(float), and 'sd':list(float) dictionaries where the distribution keys are atom label pairs (str), example: 'Li-O'.
        
    Returns:
        distribution(dict(dict)): Contains buckingham potential, 'bpp':list(float), and 'sd':list(float) dictionaries where the distribution keys are atom label pairs (str), example: 'Li-O'.      
    """   
    i = 0
    for pot in distribution.values():
        pot['bpp'] = [round(modes[i],3), round(modes[i+1],3), round(modes[i+2],3)]
        pot['sd'] = [round(sd[i],3), round(sd[i+1],3), round(sd[i+2],3)]
        i += 3
    return distribution

In [6]:
for i in range(0,4):
    distribution = update_potentials(sd_list[i], modes_list[i], distribution)
    #Runs with mode potential and returns forces
    kwargs = mode_potentials(trace)
    mode_forces = fit_data.get_forces(**kwargs)

ValueError: could not convert string to float: '...'

In [34]:
summary_data = pd.read_csv(summary_filename)
modes_list
sd_list

In [55]:
distribution = update_potentials(sd_list[0], modes_list[0], distribution)
#Runs with mode potential and returns forces
kwargs = mode_potentials(trace)
mode_forces = fit_data.get_forces(**kwargs)

NameError: name 'trace' is not defined